Step 2. 
Get the dataGet the data
Convert the data to a format you can easily manipulate (without changing the data itself).

In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib as plt
import numpy as np
import parker


In [ ]:
# Read the CSV file
dataset = pd.read_csv('Data/50_Startups.csv')
print(dataset)

In [ ]:
# Extracting possible features: R&D Spend, Administration, Marketing Spend & State
X = dataset.iloc[:, :-1].values

# Extraction labels: Profit
y = dataset.iloc[:, 4].values

In [ ]:
# Check the size and type of data (time series, sample, geographical, etc.).
print('Data set info:')
print(dataset.info())

print('States with count:')
print(dataset["State"].value_counts())

print('Data set type:')
print(type(dataset))

print('Data set description:')
print(dataset.describe())

In [ ]:
# Visualize the data
dataset.hist()

plt.show()

Create a test set, put it aside, and never look at it (no data snooping!).
A problem here is that the data set is small -> Use stratification.
Action: Make stratified test and learning sets (1 fold).
Code is to be established in your notebook.


In [ ]:
train_set, test_set = parker.shuffle_and_split_data(dataset, 0.2)

print(len(train_set))

print(len(test_set))

print(test_set)

In [ ]:
startup_with_id = dataset.reset_index() # adds an `index` column

train_set, test_set = split_data_with_id_hash(startup_with_id, 0.8, "index")
print("Stable test set based on rows index, 80% size od training set")
print(test_set)

In [ ]:
train_set, test_set = train_test_split(dataset, test_size=0.6, random_state=42)

print("test set using sklearn instead")
print(test_set)

In [ ]:
dataset["Marketing Spend"] = pd.cut(dataset["Marketing Spend"],
 bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
 labels=[1, 2, 3, 4, 5])
 
 
dataset["Marketing Spend"].value_counts().sort_index().plot.bar(rot=0, grid=True) 
plt.xlabel("marketing spend")
plt.ylabel("Number of districts")
plt.show()



In [ ]:
splitter = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
strat_splits = []
for train_index, test_index in splitter.split(dataset, dataset["Marketing Spend"]):
    strat_train_set_n = dataset.iloc[train_index]
    strat_test_set_n = dataset.iloc[test_index]
    strat_splits.append([strat_train_set_n, strat_test_set_n])